In [12]:
import pandas as pd
import copy
import random
import plotly.express as px

from datetime import date
from matplotlib import pyplot as plt

In [13]:
################### Introduction ########################
# This is a mini, non-quantum trial for implementing an algorithm that creates prototypes and schedules tests on them out of given data.
# The data is in the form of an excel file where each row specifies an order or "test" that has to be performed on a certain date with a specific kind of configuration.
# The algorithm checks for shared configurations between these test and specifies prototypes accordingly. These prototypes are then scheduled using a brute-force approach (greedy scheduling), which is blunt but makes sense here, as the set starting dates and configurations of the test heavily reduce the possible solutions.
# Instead of claiming to have an optimal solution, this algorithm primarily serves to create a starting point for the problem that can then be adjusted by hand later.

In [14]:
# Read example data
jobs = pd.read_excel('data.xlsx')

In [15]:
# Clean up data
jobs.replace({'O': 0}, inplace=True) # O in the excel means "irrelevant". We assign this property to the number 0 for easier handling later

jobs_alpha = jobs[jobs['Phase'] == 'Alpha']
jobs_beta = jobs[jobs['Phase'] == 'Beta']
jobs_gamma = jobs[jobs['Phase'] == 'Gamma']
jobs_delta = jobs[jobs['Phase'] == 'Delta']

In [16]:
# Prepare data relevant for prototypes
strip_columns = ['Part 1', 'Part 2', 'Part 3', 'Part 4', 'Part 5', 'Part 6']
prototypes_alpha = jobs_alpha[strip_columns]
prototypes_beta  = jobs_beta[strip_columns]
prototypes_gamma = jobs_gamma[strip_columns]
prototypes_delta = jobs_delta[strip_columns]

In [17]:
################### Functions ########################
# Sort jobs by number of possible prototypes
def get_num_prototypes(elem):
    return len(elem.get('prototypes'))

# Sort Jobs by starting time
def get_KW(elem):
    return elem.get('daterange')[0] + elem.get('daterange')[1]


def split_column(df_list, column):
    """
    Splits data-frames by individual column entries
    :param df_list: List of data-frames
    :param column: Name of column by which to split
    :return: Returns list of split data-frames
    """
    temp_list = []
    for elem in df_list:
        for k in elem[column].unique():
            if (k == 0) and (len(elem[column].unique()) > 1):
                pass
            else:
                temp_list.append(elem[(elem[column] == k) | (elem[column] == 0)])
    
    return temp_list

def greedy_scheduling(J, p, suffix=''):
    """
    Assigns Jobs in J to prototypes in p in the order of J. If multiple prototypes are possible, job will be assigned randomly. 
    This could be improved and made smarter but is sufficient as a proof of concept.
    :param J: List of job dictionaries
    :param p: Dictionary of prototype objects to which jobs are assigned
    :param suffix: Suffix to add to the prototype name (for duplicate prototypes)
    :return: Returns list of unassigend jobs (because all possible prototypes are already occupied)
    """
    unassigned_jobs=[]
    for i, job in enumerate(J):
        job['prototypes']
        # Check if prototypes are available
        possible_prototypes = []
        for name in job['prototypes']:
            if p[name + suffix].check_availability(job):
                possible_prototypes.append(name)
        if not possible_prototypes:
            unassigned_jobs.append(job)
        else:
            # Randomly choose one possible prototype
            final_prototype = possible_prototypes[random.randint(0, len(possible_prototypes)-1)]
            p[final_prototype + suffix].append_job(job)
    return unassigned_jobs


In [18]:
class prototype():
    """
    Prototype Class, stores schedule and information of individual prototypes
    :param name: Name of prototype
    :param df: data frame containing prototype info. Can be left empty
    """
    def __init__(self, name, df=''):
        self.name = name

        self.jobs = []
        self.occupied = []
        self.dataFrame = df
        
    def append_job(self, job):
        """
        Appends job to prototype if prototype is available during specified execution time of job
        :param job: job to append
        """
        if not self.check_availability(job):
            raise TypeError
        self.jobs.append([job['id'], job['begin'], job['duration'], job['begin'] + job['duration']])
        self.occupied += list(range(job['begin'], job['begin'] + job['duration']))
        self.occupied.sort()
    
    def check_availability(self, job):
        """
        Checks if prototype is already occupied
        :param job: job to check
        :return: Returns True if prototype is available
        """
        job_occupied = list(range(job['begin'], job['begin'] + job['duration']))
        return not any(i in job_occupied for i in self.occupied)
    

In [19]:
# Scheduling Function
def schedule(Jobs_temp, relevant_clomuns, iterations=1):
    """
    Functions wich creates a schedule for given jobs. First all jobs are clustered into similar jobs, which could run on the same prototype. 
    Then, a prototype is created for each cluster. Finally, jobs are assigned to each prototype with a greedy scheduling function, which first assigns jobs with the lowest amount of possible prototypes.
    If a job cannot be assigned because all the possible slots are already occupied, the necessary prototype is duplicated. 
    Due to the random nature of this scheduling, the output of this function can differ when called multiple times. By increasing the amount of iterations however, this randomness can be minimized.
    :param Jobs_temp: Data-frame of jobs
    :param relevant_columns: List of column names which are relevant for the prototype creation
    :param iterations: Number of types the greedy scheduling should be performed. Schedule with lowest amount of prototypes will be kept.
    :return: Returns tuple of list of jobs and dictionary of prototype objects to which the job are assigned (schedule)
    """
    
    Jobs = Jobs_temp.copy(deep=True)
    Prototypes = Jobs[relevant_columns]
    
    # Cluster similar jobs
    prototypes_reduced = [Prototypes]
    for column in Prototypes.columns:
        prototypes_reduced = split_column(prototypes_reduced, column)

    # Parse Start Date as a list of [year, calender week]
    for i in Jobs.index:
        f = Jobs['Start Date'][i]
        k = f.split('CW')[1]
        k = k.split(' ')     
        Jobs['Start Date'][i] = k[::-1]

    # Create list of jobs
    J = [{'id': i, 'Department': Jobs['Department'][i], 'Titel': Jobs['Titel'][i],'phase': Jobs['Phase'][i], 'duration': int(Jobs['Duration'][i]), 'prototypes': [], 'daterange': Jobs['Start Date'][i]} for i in Jobs.index]

    # Create prototype object for every prototype
    p = dict()
    for i, df in enumerate(prototypes_reduced):
        name = 'P{:02d}'.format(i+1)
        p[name]=prototype(name, df)
    
    # Add possible prototypes to  each job injob list
    for i in range(len(J)):
        for key, value in p.items():
            if J[i]['id'] in value.dataFrame.index:
                J[i]['prototypes'].append(value.name)
    
    # Sort Jobs by starting time
    J.sort(key=get_KW)
    # Convert calender weeks to integer numbers
    start = J[0].get('daterange')
    for i, job in enumerate(J):
        if int(job['daterange'][0]) > int(start[0]):
            # Add Schaltyear here
            J[i]['begin'] = int(job['daterange'][1]) + (52-int(start[1])) + 52*(int(job['daterange'][0])-int(start[0])-1)
        else:
            J[i]['begin'] = int(job['daterange'][1]) - int(start[1])
    
    # Sort jobs by number of possible prototypes
    J.sort(key=get_num_prototypes)
    
    p_list = []  
    
    # Try greedy sheduling multiple times and keep best result
    for j in range(iterations):
        
        p_list.append(copy.deepcopy(p))
        
        unassigned_jobs = greedy_scheduling(J, p_list[j])
        
        p_new = dict()
        m = 1
        while unassigned_jobs:
            needed_prototypes = {}
            for i in unassigned_jobs:
                l = i['prototypes']
                for k in l:
                    if k in needed_prototypes:
                        needed_prototypes[k] += 1
                    else:
                        needed_prototypes[k] = 1

            for key, value in needed_prototypes.items():
                name = key+("*"*m)
                p_new[name]=prototype(name, 'Duplicate')

            unassigned_jobs = greedy_scheduling(unassigned_jobs, p_new, ("*"*m))
            m += 1
            
        p_list[j] = p_list[j] | p_new
        p_list[j] = {k: v for k, v in p_list[j].items() if v.jobs}
            
    p = min(p_list, key=len)
    
    return J, p

In [20]:
relevant_columns = ['Part 1', 'Part 2', 'Part 3', 'Part 4', 'Part 5', 'Part 6']
J_alpha, p_alpha = schedule(jobs_alpha, relevant_columns, 5)
J_beta, p_beta = schedule(jobs_beta, relevant_columns, 5)
J_gamma, p_gamma = schedule(jobs_gamma, relevant_columns, 5)
J_delta, p_delta = schedule(jobs_delta, relevant_columns, 5)

<ipython-input-19-bc3f5ed3e386>:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [21]:
def plot_list_gen(J, p):
    """
    Parses data returned by schedule-function into list that can be plotted by plotly express
    :param J: List of job-dicionaries
    :param p: Dictionary of prototype objects
    :return: Returns list of dictionaries for each assigned job
    """
    prot_list= []
    for key, value in p.items():
        for k in value.jobs:
            job_dict = next(item for item in J if item["id"] == k[0])
            phase = job_dict['phase']
            id_num = job_dict['id']
            year = int(job_dict['daterange'][0])
            week = int(job_dict['daterange'][1])
            duration = job_dict['duration']
            titel = job_dict['Titel']
            poss_prots = job_dict['prototypes']
            department = job_dict['Department']
            if week + duration > 52:
                final_year = year + 1
                final_week = week + duration - 52
            else:
                final_year = year
                final_week = week + duration            
            prot_list.append(dict(Prototyp=value.name, Possible_prototypes=poss_prots, Duration=duration, Department=department, ID=id_num, Phase=phase, Titel=str(id_num) + "_" + titel, Start=date.fromisocalendar(year, week, 1), Finish=date.fromisocalendar(final_year, final_week, 1)))
    
    return prot_list

In [22]:
def plot_schedule(J, p, name=''):
    """
    Plots schedule with plotly
    :param J: List of job-dicionaries
    :param p: Dictionary of prototype objects
    :param name: Name that is put into title
    """
    prot_list = plot_list_gen(J, p)
    df = pd.DataFrame(prot_list)

    fig = px.timeline(df, title="Scheduling {}-Phase".format(name),x_start="Start", x_end="Finish", y="Prototyp", color="Department", hover_name='Titel', hover_data=["ID", "Phase", "Duration", "Possible_prototypes"])
    #fig = fig.full_figure_for_development(warn=False)
    #fig.layout.barmode = "group"
    fig.update_yaxes(automargin=True, categoryorder="category descending")
    fig.update_layout(height=10*len(prot_list)+300)
    #fig.update_yaxes(type="category")
    fig.show()
    
    print('Number of orders: {} \nNumber of calculated prototypes: {} \n'.format(len(J), len(p)))
    
    for key, value in p.items():
        print(key)
        display(value.dataFrame)

In [23]:
plot_schedule(J_alpha, p_alpha, 'Alpha')
#plot_schedule(J_beta, p_beta, 'Beta')
#plot_schedule(J_gamma, p_gamma, 'Gamma')
#plot_schedule(J_delta, p_delta, 'delta')

Number of orders: 37 
Number of calculated prototypes: 21 

P01


,Part 1,Part 2,Part 3,Part 4,Part 5,Part 6
0,A,A,A,0,0,0
16,A,A,A,A,A,A


P02


,Part 1,Part 2,Part 3,Part 4,Part 5,Part 6
0,A,A,A,0,0,0
62,A,A,A,D,0,0
65,A,A,A,D,0,0
69,A,A,A,D,0,0
93,A,A,A,D,0,0
96,A,A,A,D,0,0


P03


...

'Duplicate'

P03*


'Duplicate'

P04*


'Duplicate'